In [ ]:
import os
import sys
import mne
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# check some package versions for documentation and reproducability
print('Python sys', sys.version)
# print('pandas', pd.__version__)
print('numpy', np.__version__)
# print('mne_bids', mne_bids.__version__)
print('mne', mne.__version__)

In [ ]:
data_path = os.path.dirname(os.path.dirname(os.getcwd()))
data_path = os.path.join(data_path, 'data', 'source_data')
sub_data = os.path.join(data_path, 'sub01')
files = [f for f in os.listdir(sub_data) if f.endswith('.con')]
file = os.path.join(sub_data, files[0])
markers = [f for f in os.listdir(sub_data) if f.endswith('.mrk')]
marker = os.path.join(sub_data, mrks[0])

print(f'first file: {file}\nfirst marker: {marker}')

## Read Data File

In [ ]:
# # ascii
# # with open(os.path.join(sub_data, files[0]), 'rt', encoding='utf-8', errors='ignore') as f:  # read text not working
# with open(file, 'rb',) as f:  # rb -> eradbinary
#     # raw = f.read()
#     raw = np.fromfile(f, 'uint16')

# print(raw.shape)

In [ ]:
raw = mne.io.read_raw_kit(file,)
dat, times = raw[:]
chnames = raw.ch_names

Check loaded content

In [ ]:
print(raw.info)

print(f'Channel names within .con data, (n={len(raw.ch_names)}): {raw.ch_names}')
print(f'Timestamps within .con data (n={len(raw.times)}): {raw.times}')
print(f'Sampling freq within .con data: {raw.info['sfreq']}')

print(f'\ndata shape: {dat.shape}, times shape: {times.shape}')

# dat.info['chs][i]  # contains dict with info on channel

In [ ]:
# check loaded data
raw.info

In [ ]:
from itertools import compress, product

Pick MEG channels and data

In [ ]:
# meg_ch_pick = mne.pick_types(raw.info, meg=True, exclude='bads')  # only 214 instead of 128
meg_chs = [c for c in raw.info['chs'] if c['ch_name'].startswith('MEG')]  # or based on type: [c for c in raw.info['chs'] if 'MEG' in getattr(c['kind'], '_name')]
print(len(meg_chs))

meg_bool = [c['ch_name'].startswith('MEG') for c in raw.info['chs']]
megdat = dat[meg_bool, :]
print(megdat.shape)

In [ ]:
stim_bool = [c['ch_name'].startswith('STIM') for c in raw.info['chs']]
stimdat = dat[stim_bool, :]

In [ ]:
stimdat

In [ ]:
i = -10

plt.plot(dat[i, :])
plt.title(chnames[i])

plt.show()

In [ ]:
for i in np.arange(10):
    plt.plot(dat[meg_chs, :][i])

plt.show()

In [ ]:
raw.plot_sensors()

Preprocessing trial

- source: https://mne.tools/dev/auto_examples/datasets/opm_data.html

In [ ]:
raw.load_data()
raw.filter(None, 90, h_trans_bandwidth=10,)  # lfreq and hfreq at beginning
# raw.notch_filter(50, notch_widths=1)

In [ ]:
# pick random first 6 MEG channels
n_picks = 30
megpicks = np.where([c['ch_name'].startswith('MEG') and c['logno'] < n_picks for c in raw.info['chs']])[0]
# imaginary events every 10 seconds
n_events = 25
events = np.zeros((n_events, 3))
events[:, 0] = raw.info['sfreq'] * np.arange(n_events)
events[:, 2] = np.ones((n_events))
events = events.astype(int)


In [ ]:
epochs = mne.Epochs(raw, events=events, tmin=-2, tmax=5, picks=megpicks, preload=True)

In [ ]:
evoked = epochs.average()
evoked.plot()

## Read Marker

In [ ]:
# ascii
with open(marker, 'rt', encoding='utf-8', errors='ignore') as f:  # rt -> read text; 
    print(marker)
    mrk = f.read()
    

In [ ]:
mrk = mne.io.kit.read_mrk(marker,)
print(f'marker shape: {mrk.shape}')

In [ ]:
mrk